In [12]:
import numpy as np 
from sklearn.datasets import load_diabetes

In [13]:
X, y = load_diabetes(return_X_y=True)

In [14]:
diabetes = load_diabetes()

In [15]:
diabetes.feature_names

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

In [16]:
from sklearn.linear_model import LinearRegression

In [17]:
from sklearn.model_selection import  train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [18]:
y_pred = reg.predict(X_test)

In [19]:
from sklearn.metrics import r2_score

In [20]:
r2_score(y_test, y_pred)

0.4399338661568969

In [21]:
reg.coef_

array([  -9.15865318, -205.45432163,  516.69374454,  340.61999905,
       -895.5520019 ,  561.22067904,  153.89310954,  126.73139688,
        861.12700152,   52.42112238])

In [22]:
reg.intercept_

151.88331005254167

# Making own linear regression class

---
***Why Insert 1 in Code***

```python

X_train = np.insert(X_train, 0, 1, axis=1)
```
This adds a column of 1's at the beginning of X_train.
The column ensures that the intercept 
𝛽
0
β 
0
​
  is learned alongside the other coefficients during training.


Inserting 1 in the first column simplifies the implementation and allows you to calculate both the intercept (
𝛽
0
β 
0
​
 ) and feature coefficients (
𝛽
1
,
𝛽
2
,
…
β 
1
​
 ,β 
2
​
 ,…) in a unified way using matrix operations.

 ---

---

***The formula:***

```python
betas = np.linalg.inv(np.dot(X_train.T, X_train)).dot(X_train.T).dot(y_train)
```

is mathematically correct for computing the coefficients $(\beta)$ in **Ordinary Least Squares (OLS)** regression using the **normal equation**:
$
\beta = (X^T X)^{-1} X^T y
$


---


In [115]:
class My_lr:

    def __init__(self): 
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X_train, y_train):
        X_train = np.insert(X_train, 0, 1, axis=1)

        # calculate the coeffitient 
        # betas = np.linalg.inv(np.dot(X_train.T, X_train)).dot(X_train.T).dot(y_train)
        betas = np.linalg.inv(X_train.T @ X_train) @ (X_train.T @ y_train)
        self.intercept_ = betas[0]
        self.coef_ = betas[1:]
        
    def predict(self, X_test): 
        y_pred = np.dot(X_test, self.coef_) + self.intercept_
        return y_pred

In [117]:
lr = My_lr()

In [119]:
lr.fit(X_train, y_train)

In [121]:
y_pred = lr.predict(X_test)

In [123]:
r2_score(y_test,y_pred)

0.43993386615689667

The implementation is using the **analytical solution** for linear regression, also known as the **normal equation**. 

---

### **Key Characteristics of Analytical Solution:**
1. The line:  
   $
   \mathbf{y} = \mathbf{X} \cdot \beta
   $  
   where $ \mathbf{X} $is the design matrix (input features with a bias column added), and $\beta$ is the vector of coefficients (including the intercept).  

2. **Normal Equation Formula:**  
   The weights (coefficients and intercept) are computed as:  
   $
   \beta = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}
   $  

3. **Your Implementation:**  
   - The line `np.linalg.inv(X_train.T @ X_train) @ (X_train.T @ y_train)` directly computes$ (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y} $, which is the normal equation.

---

### **Steps in Your Code:**
1. **Initialization:**
   - `self.coef_` and `self.intercept_` are placeholders for the model's parameters.

2. **Adding the Bias Term:**  
   - In `fit`, you insert a column of ones into `X_train` (bias term). This is required for the intercept $( \beta_0)$ to be computed.

3. **Computing Coefficients (Betas):**  
   - Using `np.linalg.inv(X_train.T @ X_train) @ (X_train.T @ y_train)`, it solves the normal equation to compute the coefficients analytically.

4. **Prediction:**  
   - The `predict` method uses the coefficients $( \beta_1, \beta_2, \dots )$ and the intercept $( \beta_0 )$ to make predictions.

---

### **Analytical Solution vs Gradient Descent:**
- **Analytical Solution (Your Code):**
  - Computes the coefficients in a single step using matrix operations.
  - Can become computationally expensive for large datasets due to the need to compute the inverse of $( X^T X )$, which has a complexity of $ O(n^3) $.

- **Gradient Descent:**
  - Iteratively updates weights using gradients of the loss function.
  - More efficient for large datasets or high-dimensional data, as it avoids matrix inversion.

---

When working with a large dataset, gradient descent or stochastic gradient descent might be more practical. However, for small to medium-sized datasets, the analytical solution works well and is straightforward to implement.

# Linear regression using **gradient descent** :
---

In [32]:
### Gradient Descent Implementation

import numpy as np

class MyLR_GD:
    def __init__(self, learning_rate=0.01, epochs=1000): 
        self.coef_ = None
        self.intercept_ = None
        self.learning_rate = learning_rate
        self.epochs = epochs

    def fit(self, X_train, y_train):
        # Initialize weights and bias to 0
        self.coef_ = np.zeros(X_train.shape[1])
        self.intercept_ = 0

        n = X_train.shape[0]  # Number of samples

        # Gradient Descent Loop
        for _ in range(self.epochs):
            # Compute predictions
            y_pred = np.dot(X_train, self.coef_) + self.intercept_

            # Compute gradients
            dw = -(2/n) * np.dot(X_train.T, (y_train - y_pred))  # Gradient for weights
            db = -(2/n) * np.sum(y_train - y_pred)              # Gradient for bias

            # Update weights and bias
            self.coef_ -= self.learning_rate * dw
            self.intercept_ -= self.learning_rate * db

    def predict(self, X_test): 
        y_pred = np.dot(X_test, self.coef_) + self.intercept_
        return y_pred



### How the code relates:
1. The gradient for $w$:
   ```python
   dw = -(2/n) * np.dot(X_train.T, (y_train - y_pred))
   ```
   Matches the vectorized formula:
   $
   \frac{\partial J}{\partial w} = -\frac{2}{n} X^T (y - \hat{y})
   $

2. The gradient for $b$:
   ```python
   db = -(2/n) * np.sum(y_train - y_pred)
   ```
   Matches:
   $
   \frac{\partial J}{\partial b} = -\frac{2}{n} \sum_{i=1}^n \left( y_i - \hat{y}_i \right)
   $

---

### Step-by-Step Explanation

#### **Initialization:**
- `self.coef_` is initialized to a zero vector with the same length as the number of features in the training data.
- `self.intercept_` is set to 0.
- `learning_rate` $( \eta )$: Controls the step size for each iteration.
- `epochs`: Defines how many iterations of gradient descent to perform.

---

#### **Training (fit method):**

1. **Initial Setup:**
   - `n` is the number of training samples, used to scale the gradients.
   - Loop for a specified number of `epochs`.

---

2. **Predictions:**
   - Predictions are computed as:  
     $
     \hat{y} = X \cdot w + b
     $  
     Here:
     - $ X $ is the input matrix.
     - $ w $ is the weights vector.
     - $ b $ is the bias term (intercept).

---

3. **Compute Gradients:**
   - The gradient of the **loss function** (Mean Squared Error, MSE) with respect to:
     - **Weights** $() w )$:  
       $
       \frac{\partial \text{Loss}}{\partial w} = -\frac{2}{n} X^T (y - \hat{y})
       $
     - **Bias** $( b )$:  
       $
       \frac{\partial \text{Loss}}{\partial b} = -\frac{2}{n} \sum (y - \hat{y})
       $
   - These gradients measure how much the loss would change if the weights or bias were updated slightly.

---

4. **Update Weights and Bias:**
   - Update rule for gradient descent:
     $
     w := w - \eta \cdot \frac{\partial \text{Loss}}{\partial w}
     $
     $
     b := b - \eta \cdot \frac{\partial \text{Loss}}{\partial b}
     $
   - Here, $ \eta $ (learning rate) determines how big a step to take in the direction of the negative gradient.

---

5. **Repeat:**
   - This process is repeated for `epochs` iterations, allowing the weights and bias to converge to values that minimize the loss function.

---

#### **Prediction (predict method):**
- Once the model is trained, predictions for new data $( X_{\text{test}} )$ are made using:
  $
  \hat{y} = X_{\text{test}} \cdot w + b
  $

---

### Comparison with Analytical Solution Code:

| **Aspect**            | **Analytical Solution**                            | **Gradient Descent**                        |
|------------------------|---------------------------------------------------|---------------------------------------------|
| **Approach**           | Computes weights in one step using a closed-form formula. | Iteratively updates weights using gradients. |
| **Complexity**         | $ O(n^3) $, due to matrix inversion.            | $\ O(n \cdot m \cdot \text{epochs})$, scales better for large data. |
| **When to Use?**       | Small to medium datasets.                         | Large datasets or high-dimensional data.    |
| **Numerical Stability**| Can be unstable if $ X^T X $ is nearly singular. | More stable and flexible.                   |

---

### Example Usage:

In [39]:
# Sample Dataset
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2.2, 2.8, 4.5, 4.9, 5.8])

# Gradient Descent Linear Regression
lr = MyLR_GD(learning_rate=0.01, epochs=1000)
lr.fit(X, y)

# Predictions
y_pred = lr.predict(X)

# Coefficients and Intercept
print("Weights (coef_):", lr.coef_)
print("Intercept (intercept_):", lr.intercept_)
print("Predictions:", y_pred)

Weights (coef_): [0.93863415]
Intercept (intercept_): 1.2188279693304327
Predictions: [2.15746212 3.09609627 4.03473043 4.97336458 5.91199873]
